In [28]:
class object_frut:
    def __init__(self,img,value,typ,color):
        self.img = img
        self.value= value
        self.type = typ
        self.color = color
        
    def getType(self):
        return self.type
    
    def getValue(self):
        return self.value
    
    def getImg(self):
        return self.img

    def getColor(self):
        return self.color

In [29]:
class player:
    def __init__(self,hit,ttl,life):
        self.ttl = ttl
        self.life = life
        self.hit = hit
        
    def getStatus(self):
        return self.hit,self.ttl,self.life
    
    def setStatus(self,ttl,hit,life,typehit='frut'):
        if self.hit + hit < 0 :self.hit = 0
        else: self.hit += hit 
        
        if typehit == 'bomba':
            life = 1
        
        self.ttl += ttl
        self.life -= life
    
    def getHit(self):
        return self.hit

In [30]:
class speed():
    
    def __init__(self,speed=[0,10],level=0):
        self.sp = speed
        self.level = level
        
    def calSpeed(self):
        self.sp[0] = - self.sp[0]

    
    def setSpeed(self,nHit):
        
        speed = [[0, 10],[0, 15],[0, 25]]
        
        self.sp = speed[0]

        if nHit >= 300:
            x = random_number(-20,20,1)
            self.sp = [x,20]
            self.level = 4 
        if nHit < 300:
            self.sp = speed[2]
            self.level = 3
        if nHit < 150:
            self.sp = speed[1]
            self.level = 2
        if nHit < 75:
            self.sp = speed[0]
            self.level = 1
  
    def getSpeed(self):
        return self.sp
    
    def getLevel(self):
        return self.level

In [31]:
def load_fruts(path):
    
    from pygame.image import load
    yellow = (253,255,164)
    burgundy = (117,12,12)
    green = (250,129,246)
    red = (118,249,148)
    black = (0,0,0)
    
    try:
        
        banana = object_frut(load(path+"banana.png"),1,'frut',yellow)
        cereja = object_frut(load(path+"cereja.png"),2,'frut',burgundy)
        melancia = object_frut(load(path+"melancia.png"),5,'frut',green)
        morango = object_frut(load(path+"morango.png"),10,'frut',red)
        bomba = object_frut(load(path+"bomba.png"),-50,'bomba',black)
        
    except Exception as inst:
        print(inst)
        raise inst
    
    return banana,cereja,melancia,morango,bomba
        
def test_load_fruts():
    
    import pytest
    
    with pytest.raises(Exception,match='Unsupported image format'):
        load_fruts("imgs/")

In [32]:
def random_number(min_value,max_value,s):
    
    from numpy.random import randint
    
    return randint(low=min_value, high=max_value, size=s)

In [33]:
def random_frut(list_fruts):
    
    
    Objfrut = list_fruts[int(random_number(0,len(list_fruts),1))]
    
    ballrect = Objfrut.getImg().get_rect(center = (int(random_number(Objfrut.getImg().get_width()//2+128,height-Objfrut.getImg().get_width()/2,1)),32))
    
    return Objfrut, ballrect

In [34]:
def score_update(ttl=-1,hit=-1,font='rogfonts',size=15,color=(115,84,201),level=0):
    
    from pygame.font import SysFont
     
    txt_ttl = SysFont(font, size).render(str(ttl), True, color)
    txt_hit = SysFont(font, size).render(str(hit), True, color)
    
    lv = SysFont(font, size).render('Lv. '+str(level), True, color)
    
    return txt_ttl, txt_hit,lv

In [35]:
def calLife(life_p,screen,heart):
    
    x=6
    y=20
    for n in range( life_p):
        screen.blit(heart,(x,y))
        x+=27

In [36]:
def setscenario(path):
    
    from pygame.image import load
    
    score = load(path+"Score.png")
    heart = load(path+"heart.png")
    backgroud = load(path+"back.png")
    level = load(path+"level.png")
    return score, heart, backgroud, level

In [37]:
def detection(image):
    
    import cv2
    import numpy as np
    from math import sqrt

    faceCascade1 = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    faceCascade2 = cv2.CascadeClassifier('haarcascade_smile.xml')
    faceCascade3 = cv2.CascadeClassifier('haarcascade_eye.xml')

    X = 0
    Y = 0
    smile_bool = False
    
    face = faceCascade1.detectMultiScale(
        image,
        minNeighbors=5,
        minSize=(200, 200),
        maxSize=(300,300)
    )
 
    smile = faceCascade2.detectMultiScale(
        image,
        minNeighbors=30,
        minSize=(80, 80),
        maxSize=(200,200)
    )
    
    eye = faceCascade3.detectMultiScale(
        image,
        minNeighbors=40,
        minSize=(30, 30),
        maxSize=(50,50)
    )
    
    for (x, y, w, h) in eye:
            cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 255), 4)

    for (x, y, w, h) in face:
        x -= 20
        y += 20
        w += 20
        h += 20
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 4)
        cv2.line(image, (x,y+h//2), (0,y+h//2), (0,0,255), 2)
        cv2.line(image, (x+w,y+h//2), (image.shape[1],y+h//2), (0,0,255), 2)

        cv2.line(image, (x+w//2,y), (x+w//2,0), (0,0,255), 2)
        cv2.line(image, (x+w//2,y+h), (x+w//2,image.shape[0]), (0,0,255), 2)

        for (x1, y1, w1, h1) in smile:
            try:
                dist = sqrt(pow((x1+w1//2)-(x+w//2),2)+pow((y1+h1//2)-(y+h//2),2))
                
                if (dist) < 90:
                    cv2.rectangle(image, (x1, y1), (x1+w1, y1+h1), (255, 0, 0), 2)
                    smile_bool = True
                    X = (x1+w1//2)
                    Y = (y1+h1//2)
                
            except:
                  print("can't discover smile")
    
    
    #cv2.putText(image, '.', (X,Y), cv2.FONT_HERSHEY_SIMPLEX , 1, (0,0,0), 2, cv2.LINE_AA) 
    
    return image,bool(len(face)),bool(len(eye)),smile_bool,X,Y
    

In [38]:
def hit_objet(X1,Y1,X2,Y2):
    
    from math import sqrt
    
    dist = sqrt(pow((X2)-(X1),2)+pow((Y2)-(Y1),2))
    
    if dist < 110:

        return True
    return False

In [39]:
def hit_show(value=0,font='rogfonts',size=30,color=(255,255,255)):
    
    from pygame.font import SysFont
    if value > 0:
        value = str("+"+str(value))
    return SysFont(font, size).render(str(value), True, color)

In [40]:
class CalHit():
    def __init__(self,time=0,x=0,y=0):
        self.frut = frut
        self.time = time
        self.x = x
        self.y = y
        
    def getFrut(self):
        return frut
    
    def getTime(self):
        return self.time
    
    def getX(self):
        return self.x
    
    def getY(self):
        return self.y
    
    def setTime(self,value):
        self.time -= value
    
    def setY(self,y):
        self.y -= y

In [41]:
import sys, pygame, cv2

size = width, height = 800, 520
cap = cv2.VideoCapture(0)

value_hit = 0   

pygame.init() 

list_fruts = load_fruts("imgs/")
score, heart, backgroud, level = setscenario("imgs_scenario/")

frut, ballrect = random_frut(list_fruts)

if frut: 
    screen = pygame.display.set_mode(size)
else: 
    sys.exit()

ply = player(0,0,4)    
Hit = CalHit()
sp = speed()

while frut:
    

    
    for event in pygame.event.get():
        if event.type == pygame.QUIT: 
            pygame.quit()
            sys.exit()
                 
    sp.setSpeed(ply.getHit())

    ballrect = ballrect.move(sp.getSpeed())
    ttl_p, hit_p, life_p = ply.getStatus()
    ttl, hit,lv = score_update(ttl_p, hit_p,level=sp.getLevel())

    _, frame = cap.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    image, face, eye, smile, X1,Y1 = detection(frame)
        
    if smile:    
        X2 = int(680-ballrect.x+(frut.getImg().get_size()[0]))
        Y2 = int(ballrect.y+frut.getImg().get_size()[0]/2)
        
        if hit_objet(X1,Y1,X2,Y2):
             
            value_hit = hit_show(value=frut.getValue(),color=frut.getColor())
            Hit = CalHit(5,ballrect.x,ballrect.y)

            ply.setStatus(1,frut.getValue(),0,typehit=frut.getType())
            frut, ballrect = random_frut(list_fruts)
    if ballrect.left < 0 or ballrect.right > width:
        speed.calSpeed()
                        
    if ballrect.top < 0 or ballrect.bottom > height:
        
        if frut.getType() == 'bomba' and eye:
            ply.setStatus(1,0,1)
            
        frut, ballrect = random_frut(list_fruts)
        ply.setStatus(1,0,0)
    
    surf = pygame.surfarray.make_surface(image)
    image = pygame.transform.rotate(surf, 270)
    
    screen.blit(backgroud,(0,0))
    screen.blit(image,(128,0))
    screen.blit(score,(-5,-10))
    screen.blit(level,(-10,100))
    screen.blit(ttl,(10,65))
    screen.blit(hit,(65,65))
    screen.blit(lv,(30,140))
    screen.blit(frut.getImg(), ballrect)
    
    
    if Hit.getTime():        
        screen.blit(value_hit,(Hit.getX(),Hit.getY()))
        Hit.setTime(1)
    Hit.setY(20)
        
    calLife(life_p,screen,heart)
    pygame.display.flip()
        
cap.release()
cv2.destroyAllWindows()
del(cap)

SystemExit: 